In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict


In [2]:
file_path = "../Data/Raw/"
data_csv_list = os.listdir(file_path)
data_csv_list

[]

In [3]:
lab_raw_df = pd.read_csv(file_path+"/"+"loinc.csv")
lab_raw_df

,STUDY_ID,VAR_TIME,NAME,VALUE_IF_TYPE_IS_NUMERIC,VALUE_IF_TYPE_IS_CODED_CODE,CODE_NAME,LOINC_CODE,LOINC_CODENAME
0,19492,04NOV2016,Cholesterol SerPl Qn,170.000,NaN,NaN,2093-3,Cholesterol [Mass/volume] in Serum or Plasma
1,19492,04NOV2016,Cholesterol/HDL SerPl Qn,2.000,NaN,NaN,9830-1,Cholesterol.total/Cholesterol in HDL [Mass Rat...
2,19492,04NOV2016,Est Avg Glucose Bld mg/dL,108.000,NaN,NaN,27353-2,Glucose mean value [Mass/volume] in Blood Esti...
3,19492,04NOV2016,Est Avg Glucose Bld mmol/L,6.000,NaN,NaN,53553-4,Glucose mean value [Moles/volume] in Blood Est...
4,19492,04NOV2016,Glucose Ser P Fast,94.000,NaN,NaN,1558-6,Fasting glucose [Mass/volume] in Serum or Plasma
...,...,...,...,...,...,...,...,...
488067,308250127,20NOV2019,Cortisol Free 24H Ur Qn,30.500,NaN,NaN,2147-7,Cortisol Free [Mass/time] in 24 hour Urine
488068,308250127,20NOV2019,Creatinine 24H Ur Qn,1.690,NaN,NaN,2162-6,Creatinine [Mass/time] in 24 hour Urine
488069,308250127,20NOV2019,Volume Ur Collected-Chem,5200.000,NaN,NaN,3167-4,Volume of 24 hour Urine
488070,308250127,16JAN2020,Body Weight,72.122,NaN,NaN,3142-7,Body weight Stated


These are the different type of lab tests that we narrowed our focus on too. 

In [4]:
hdl_of_interest = ["Cholesterol in HDL [Mass/volume] in Serum or Plasma"]

ldl_of_interest = [
"Cholesterol in LDL [Mass/volume] in Serum or Plasma by calculation",
"Cholesterol in LDL [Mass/volume] in Serum or Plasma by Direct assay"]

glucose_of_interest = [
"Glucose [Mass/volume] in Serum or Plasma",
"Glucose [Mass/volume] in Blood"]

hemoglobin_of_interest = [
"Hemoglobin [Mass/volume] in Arterial blood",
"Hemoglobin [Mass/volume] in Blood",
"Hemoglobin [Mass/volume] in Blood by calculation"]


In [5]:
glucose_lab_df = lab_raw_df[lab_raw_df['LOINC_CODENAME'].isin(glucose_of_interest)]
ldl_lab_df = lab_raw_df[lab_raw_df['LOINC_CODENAME'].isin(ldl_of_interest)]
hdl_lab_df = lab_raw_df[lab_raw_df['LOINC_CODENAME'].isin(hdl_of_interest)]
hemo_lab_df = lab_raw_df[lab_raw_df['LOINC_CODENAME'].isin(hemoglobin_of_interest)]

In [6]:
# print("Medidan number of collection: " , glucose_lab_df.groupby(["STUDY_ID", "LOINC_CODENAME"]).size().median())
# print("Average number of collection: " , glucose_lab_df.groupby(["STUDY_ID", "LOINC_CODENAME"]).size().mean())
# print("Max number of collection: " , glucose_lab_df.groupby(["STUDY_ID", "LOINC_CODENAME"]).size().max())

# glucose_lab_df.groupby(["STUDY_ID", "LOINC_CODENAME"]).size().hist()
# glucose_lab_df.groupby(["STUDY_ID", "LOINC_CODENAME"]).size()

In [7]:
glucose_lab_df.groupby(["STUDY_ID", "LOINC_CODENAME"]).size().idxmax()

(25827829, 'Glucose [Mass/volume] in Blood')

In [8]:
glucose_lab_df[glucose_lab_df["STUDY_ID"] == 25827829]

,STUDY_ID,VAR_TIME,NAME,VALUE_IF_TYPE_IS_NUMERIC,VALUE_IF_TYPE_IS_CODED_CODE,CODE_NAME,LOINC_CODE,LOINC_CODENAME
381455,25827829,18JAN2016,Glucose Bld Qn (POC),196.0,NaN,NaN,2339-0,Glucose [Mass/volume] in Blood
381456,25827829,18JAN2016,Glucose Bld Qn (POC),200.0,NaN,NaN,2339-0,Glucose [Mass/volume] in Blood
381466,25827829,08FEB2016,Glucose SerPl Qn,286.0,NaN,NaN,2345-7,Glucose [Mass/volume] in Serum or Plasma
381484,25827829,16MAR2016,Glucose SerPl Qn,319.0,NaN,NaN,2345-7,Glucose [Mass/volume] in Serum or Plasma
381520,25827829,18MAR2016,Glucose SerPl Qn,253.0,NaN,NaN,2345-7,Glucose [Mass/volume] in Serum or Plasma
...,...,...,...,...,...,...,...,...
386351,25827829,29DEC2017,Glucose Bld Qn (POC),307.0,NaN,NaN,2339-0,Glucose [Mass/volume] in Blood
386375,25827829,11JUL2019,Glucose SerPl Qn,291.0,NaN,NaN,2345-7,Glucose [Mass/volume] in Serum or Plasma
386419,25827829,19SEP2019,Glucose SerPl Qn,338.0,NaN,NaN,2345-7,Glucose [Mass/volume] in Serum or Plasma
386467,25827829,09JAN2020,Glucose SerPl Qn,87.0,NaN,NaN,2345-7,Glucose [Mass/volume] in Serum or Plasma


Turn time of study (string) into datetime object

In [12]:
import datetime
def make_date_time_obj(string):
    dict_month = {"JAN": 1, "FEB": 2, "MAR": 3, "APR": 4, "MAY": 5, "JUN": 6, 
                  "JUL": 7, "AUG": 8 ,"SEP": 9 , "OCT": 10, "NOV": 11 ,"DEC": 12}
    try:
        day = int(string[0:2])
        month = string[2:5]
        year = int(string[5:])
        
        month = dict_month[month]
        return datetime.datetime(year, month, day)
        
    except:
        print("An exception occurred")
        print(string)

    

In [13]:
#glucose_lab_df
#ldl_lab_df
#hdl_lab_df
#hemo_lab_df

hdl_lab_df["VAR_TIME"] = hdl_lab_df["VAR_TIME"].apply(lambda x: make_date_time_obj(x))
hdl_lab_df = hdl_lab_df.sort_values(by=['STUDY_ID', "VAR_TIME"])

ldl_lab_df["VAR_TIME"] = ldl_lab_df["VAR_TIME"].apply(lambda x: make_date_time_obj(x))
ldl_lab_df = ldl_lab_df.sort_values(by=['STUDY_ID', "VAR_TIME"])

glucose_lab_df["VAR_TIME"] = glucose_lab_df["VAR_TIME"].apply(lambda x: make_date_time_obj(x))
glucose_lab_df = glucose_lab_df.sort_values(by=['STUDY_ID', "VAR_TIME"])

hemo_lab_df["VAR_TIME"] = hemo_lab_df["VAR_TIME"].apply(lambda x: make_date_time_obj(x))
hemo_lab_df = hemo_lab_df.sort_values(by=['STUDY_ID', "VAR_TIME"])

/var/folders/72/zllm1_k55v9809qz_mgkcq6m0000gn/T/ipykernel_4304/3643373639.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdl_lab_df["VAR_TIME"] = hdl_lab_df["VAR_TIME"].apply(lambda x: make_date_time_obj(x))
/var/folders/72/zllm1_k55v9809qz_mgkcq6m0000gn/T/ipykernel_4304/3643373639.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldl_lab_df["VAR_TIME"] = ldl_lab_df["VAR_TIME"].apply(lambda x: make_date_time_obj(x))
/var/folders/72/zllm1_k55v9809qz_mgkcq6m0000gn/T/ipykernel_4304/3643373639.py:12: Se

In [14]:
def time_series_from_dfid(df_id, length_of_seq, days_between_collection):
    '''
    Build a time series of size length_of_seq (potentially with missing values) from df_id and id_. 
    
    df_id should be sorted with respect to the date time object. We will use the first row as the starting point of
    time series. Then every subsequent value will be mapped based on the delta between the current datetime object
    and the starting point. We divide the delta by the length_of_seq to find the index of the values 
    along the time series. If there are multiple mapping to the same index, we take the average of these values.
    
    Parameters: 
    
    df_id -- dataframe -- : Contains all the date time object and the values assoicated with one patient id_
    id_ -- int -- : patient id_
    length_of_seq -- int -- : length of the time series we're trying to build
    
    
    Return: Time series
    '''
    first_day = df_id["VAR_TIME"][0]
    value_time_series_dict = {i: [] for i in range(length_of_seq)}
    for index, row in df_id.iterrows():
        current_day = row["VAR_TIME"] #datetime object
        current_value = row["VALUE_IF_TYPE_IS_NUMERIC"] #value

        delta_day = (current_day - first_day).days #get days only
        index_of_value = delta_day // days_between_collection
        if 0 <= index_of_value < length_of_seq:
            value_time_series_dict[index_of_value].append(current_value)
    
    time_series_arr = np.empty(length_of_seq)

    time_series_arr[:] = np.NaN
    for index in value_time_series_dict:
        if value_time_series_dict[index]:
            time_series_arr[index] = np.array(value_time_series_dict[index]).mean()

    return time_series_arr

In [15]:
def time_series_dict_from_lab(lab_df):
    
    """
    Build a dictionary where the key is the patient id and value is the time series of lab measurements.
    """
    
    length_of_seq = 6 #length of times series
    days_between_collection = 180 # ~~ 365*3/12: 3 years worth of lab data, length 12 time series sequence

    study_id_arr= lab_df.STUDY_ID.unique()
    id_time_series_dict = defaultdict(list)
    
    for id_ in study_id_arr:
        df_id = lab_df[lab_df["STUDY_ID"] == id_].sort_values(by = ["VAR_TIME"]).reset_index()
        time_series_id = time_series_from_dfid(df_id, length_of_seq, days_between_collection)
        id_time_series_dict[id_] = time_series_id

    return id_time_series_dict

Make dataframes for each specific type of lab test

In [16]:
# hdl_of_interest = ["Cholesterol in HDL [Mass/volume] in Serum or Plasma"]

# ldl_of_interest = [
# "Cholesterol in LDL [Mass/volume] in Serum or Plasma by calculation",
# "Cholesterol in LDL [Mass/volume] in Serum or Plasma by Direct assay"]

# glucose_of_interest = [
# "Glucose [Mass/volume] in Serum or Plasma",
# "Glucose [Mass/volume] in Blood"]

# hemoglobin_of_interest = [
# "Hemoglobin [Mass/volume] in Arterial blood",
# "Hemoglobin [Mass/volume] in Blood",
# "Hemoglobin [Mass/volume] in Blood by calculation"]



ldl_lab_calculation_df = ldl_lab_df[ldl_lab_df["LOINC_CODENAME"] == ldl_of_interest[0]]
ldl_lab_direct_df = ldl_lab_df[ldl_lab_df["LOINC_CODENAME"] == ldl_of_interest[1]]

glucose_lab_serum_df = glucose_lab_df[glucose_lab_df["LOINC_CODENAME"] == glucose_of_interest[0]]
glucose_lab_blood_df = glucose_lab_df[glucose_lab_df["LOINC_CODENAME"] == glucose_of_interest[1]]

hemo_lab_arterial_df=hemo_lab_df[hemo_lab_df["LOINC_CODENAME"] == hemoglobin_of_interest[0]]
hemo_lab_blood_df = hemo_lab_df[hemo_lab_df["LOINC_CODENAME"] == hemoglobin_of_interest[1]]
hemo_lab_blood_calculation_df = hemo_lab_df[hemo_lab_df["LOINC_CODENAME"] == hemoglobin_of_interest[2]]


In [17]:
print("The number of study id of", ldl_of_interest[0], "is", ldl_lab_calculation_df.STUDY_ID.unique().size)
print("The number of study id of", ldl_of_interest[1], "is", ldl_lab_direct_df.STUDY_ID.unique().size)

print("The number of study id of", glucose_of_interest[0], "is", glucose_lab_serum_df.STUDY_ID.unique().size)
print("The number of study id of", glucose_of_interest[1], "is", glucose_lab_blood_df.STUDY_ID.unique().size)

print("The number of study id of", hemoglobin_of_interest[0], "is", hemo_lab_arterial_df.STUDY_ID.unique().size)
print("The number of study id of", hemoglobin_of_interest[1], "is", hemo_lab_blood_df.STUDY_ID.unique().size)
print("The number of study id of", hemoglobin_of_interest[2], "is", hemo_lab_blood_calculation_df.STUDY_ID.unique().size)



The number of study id of Cholesterol in LDL [Mass/volume] in Serum or Plasma by calculation is 1109
The number of study id of Cholesterol in LDL [Mass/volume] in Serum or Plasma by Direct assay is 88
The number of study id of Glucose [Mass/volume] in Serum or Plasma is 1232
The number of study id of Glucose [Mass/volume] in Blood is 346
The number of study id of Hemoglobin [Mass/volume] in Arterial blood is 2
The number of study id of Hemoglobin [Mass/volume] in Blood is 1173
The number of study id of Hemoglobin [Mass/volume] in Blood by calculation is 201


Only make time series from the 3 lab tests with data on at least 1000 patients

In [18]:
#[n_samples, seq_len, n_features]

#glucose_lab_serum_df #1232
#ldl_lab_calculation_df #1109
#hemo_lab_blood_df #1173

glucose_lab_serum_time_series_dict = time_series_dict_from_lab(glucose_lab_serum_df)
ldl_lab_calculation_time_series_dict = time_series_dict_from_lab(ldl_lab_calculation_df)
hemo_lab_blood_time_series_dict = time_series_dict_from_lab(hemo_lab_blood_df)


Concat time series

In [19]:
time_series_concat = defaultdict(list)

list_of_lab_time_series = [glucose_lab_serum_time_series_dict, 
                           ldl_lab_calculation_time_series_dict, 
                           hemo_lab_blood_time_series_dict]

largest_study_lab_id = np.argmax([len(dic) for dic in list_of_lab_time_series]) #get the lab with max_id
largest_study_lab_id 

largest_study_lab = list_of_lab_time_series[largest_study_lab_id]
for id_ in largest_study_lab:
    present_all_flag = all(id_ in d for d in list_of_lab_time_series)
    if present_all_flag:
        for lab in list_of_lab_time_series:
            time_series_concat[id_].append(lab[id_])
        

time_series_concat


defaultdict(list,
            {19492: [array([ 85.5, 107. ,  98. ,   nan,   nan,   nan]),
              array([87., nan, 80., nan, 94., nan]),
              array([13.9, 12.6, 13.8,  nan,  nan,  nan])],
             20956: [array([ 96., 103.,  nan,  nan, 100.,  nan]),
              array([79., 67., nan, nan, 69., nan]),
              array([14.6,  nan,  nan,  nan,  nan,  nan])],
             25874: [array([101.,  75., 103., 113.,  92.,  89.]),
              array([110.,  nan,  nan,  nan,  nan,  nan]),
              array([11.6 ,   nan, 12.3 , 11.55, 11.45, 11.6 ])],
             40532: [array([ 89.66666667,  99.66666667,  93.        ,  91.        ,
                      84.        , 149.        ]),
              array([ 75.,  77.,  83.,  nan,  nan, 122.]),
              array([12.28571429, 13.45      , 13.425     , 13.63333333, 14.1       ,
                     13.52      ])],
             41155: [array([109.5,  96. ,  92. ,  97.5, 108. , 116. ]),
              array([64.5,  nan,  nan,

In [21]:
number_of_sample = len(time_series_concat)
length_of_seq = 6 #length of times series
n_features = 3 # number of lab tests we're tracking

time_series_array_brits = np.zeros((number_of_sample, length_of_seq, n_features))

for i, id_ in enumerate(time_series_concat.keys()):
    array_ts = time_series_concat[id_]
    temp_arr = []
    for np_array in array_ts:
        temp_arr.append(np_array.reshape(np_array.size, 1))
    ts_concat = np.concatenate(temp_arr, axis = 1)
    time_series_array_brits[i] = ts_concat

In [22]:
time_series_array_brits.shape

(991, 6, 3)

In [25]:
from pypots.imputation import BRITS

rnn_hidden_size = 32
brits = BRITS(n_steps = length_of_seq, n_features = n_features, rnn_hidden_size= rnn_hidden_size)


Model initialized successfully. Number of the trainable parameters: 10784


In [26]:
brits.fit(time_series_array_brits)

epoch 0: training loss 153.9172
epoch 1: training loss 150.4611
epoch 2: training loss 146.9895
epoch 3: training loss 142.7644
epoch 4: training loss 138.4194
epoch 5: training loss 135.1792
epoch 6: training loss 132.0543
epoch 7: training loss 129.2878
epoch 8: training loss 126.7620
epoch 9: training loss 124.5550
epoch 10: training loss 122.1562
epoch 11: training loss 119.8801
epoch 12: training loss 117.9535
epoch 13: training loss 115.3034
epoch 14: training loss 113.4270
epoch 15: training loss 111.2768
epoch 16: training loss 108.9153
epoch 17: training loss 106.4600
epoch 18: training loss 103.9514
epoch 19: training loss 101.2376
epoch 20: training loss 98.7299
epoch 21: training loss 96.1568
epoch 22: training loss 93.4434
epoch 23: training loss 91.5353
epoch 24: training loss 89.5279
epoch 25: training loss 87.6199
epoch 26: training loss 86.1538
epoch 27: training loss 84.4104
epoch 28: training loss 83.1420
epoch 29: training loss 81.5237
epoch 30: training loss 79.780

In [27]:
time_series_array_imputed = brits.impute(time_series_array_brits)

In [28]:
time_series_array_imputed

array([[[ 85.5      ,  87.       ,  13.9      ],
        [107.       , 100.12488  ,  12.6      ],
        [ 98.       ,  80.       ,  13.8      ],
        [ 95.02008  ,  94.046326 ,  13.148533 ],
        [ 89.55194  ,  94.       ,  12.0482235],
        [ 53.121475 ,  51.980015 ,   7.903573 ]],

       [[ 96.       ,  79.       ,  14.6      ],
        [103.       ,  67.       ,  12.180351 ],
        [ 94.95173  ,  93.99718  ,  13.146215 ],
        [ 94.97539  ,  94.03845  ,  13.152522 ],
        [100.       ,  69.       ,  11.942497 ],
        [ 53.118378 ,  51.97947  ,   7.903849 ]],

       [[101.       , 110.       ,  11.6      ],
        [ 75.       ,  80.92749  ,  11.409076 ],
        [103.       ,  99.01633  ,  12.3      ],
        [113.       , 103.72459  ,  11.55     ],
        [ 92.       ,  89.157425 ,  11.45     ],
        [ 89.       ,  83.91211  ,  11.6      ]],

       ...,

       [[ 87.       ,  73.       ,   8.3      ],
        [ 88.57858  ,  87.31882  ,  13.9      ],
 

In [29]:
time_series_imputed_df = pd.DataFrame([list(l) for l in time_series_array_imputed]).stack().apply(pd.Series).reset_index(1, drop=True)
time_series_imputed_df.index.name = 'STUDY_ID'
time_series_imputed_df.columns = ["glucose_lab", "ldl_lab", "hemo_lab"] # Refer to list_of_lab_time_series 
time_series_imputed_df

,glucose_lab,ldl_lab,hemo_lab
STUDY_ID,,,
0,85.500000,87.000000,13.900000
0,107.000000,100.124878,12.600000
0,98.000000,80.000000,13.800000
0,95.020081,94.046326,13.148533
0,89.551941,94.000000,12.048223
...,...,...,...
990,88.000000,91.458969,14.300000
990,94.989746,94.018959,13.142403
990,57.000000,75.322372,14.600000


In [30]:
time_series_imputed_df = time_series_imputed_df.reset_index()
index_study_id = {i: study_id for i, study_id in enumerate(time_series_concat.keys()) }
time_series_imputed_df["STUDY_ID"] = time_series_imputed_df["STUDY_ID"].replace(index_study_id)
time_series_imputed_df

,STUDY_ID,glucose_lab,ldl_lab,hemo_lab
0,19492,85.500000,87.000000,13.900000
1,19492,107.000000,100.124878,12.600000
2,19492,98.000000,80.000000,13.800000
3,19492,95.020081,94.046326,13.148533
4,19492,89.551941,94.000000,12.048223
...,...,...,...,...
5941,301575470,88.000000,91.458969,14.300000
5942,301575470,94.989746,94.018959,13.142403
5943,301575470,57.000000,75.322372,14.600000
5944,301575470,86.346077,85.355927,12.057415


# Exporting imputed lab series

In [31]:
cwd = os.getcwd()
path = cwd + "../Data/Proccessed/ImputedLab.csv"
time_series_imputed_df.to_csv(path)